# Capstone Project Notebook -Clustering stations in the Italian region of Lazio
### IBM Data Scientist Certified Capstone by IBM/Coursera

## Table of contents
* [Introduction](#introduction)
* [Business Problem](#businessproblem)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction<a name="introduction"></a>

This project is a "Capstone Project" of Coursera - "IBM Certified Data Scientist" program.
The project is focused on the railway stations located in the Italian region of Lazio, a territory populated by 5.9 million people, visited by more than 12 million tourists each year and served by 165 train stations (41 of them inside the city of Rome).
Rete Ferroviaria Italiana is the owner of the stations, and classifies them as “bronze”, “silver”, “gold” and “platinum”, depending on the services offered at the venue, to both travelers and non-travelers.
Goal of the project is to use data gathered from Foursquare and other sources to classify the stations independently and understand if some stations should deserve a different status, according to the characteristics of the nearby area.

## Business Problem <a name="businessproblem"></a>

RFI train stations underwent a significant transformation, from travel hubs to meeting centers and places of aggregation, going beyond their original role of serving travelers by offering diverse services to travelers and non-travelers as well, such as catering services, parking places, retail stores, coworking places and more.
The service offering of a station depends on its class, that goes from “bronze” to “silver”, “gold” and “platinum”.
Goal of this study is to use open data about the 165 train stations in the Italian region of Lazio to categorize them depending on a series of factors, such as the number of check-ins and reviews, and characteristics of the nearby area such as traffic and population, and the number of significant venues in a radius of 1 km from the station, such as number of restaurants, museums, universities, cafes, professional buildings, hotels, shops, gyms and more.
This categorization will be useful to compare the current classification made by RFI with the one deriving from the above parameters, to understand if some station should deserve a different status, such as a promotion or a demotion.
The main target audience will be, therefore, RFI management. 

## Data <a name="data"></a>

The table below reports the type of data needed and their sources.

| Data | Source | Last updated |
| --- | --- | --- |
| List of RFI train stations in the Lazio region, with location and classification | http://www.rfi.it/rfi/LINEE-STAZIONI-TERRITORIO/Nelle-regioni/Lazio | 2019 |
| Number of check-ins and reviews of each station | FourSquare API | Daily updates |
| Top 100 venues in a 1000 meters range of each station, categorized by high-level groups | FourSquare API | Daily updates |
| Population and density of the neighborhoods hosting the stations, inside Rome | https://it.wikipedia.org/wiki/Municipi_di_Roma | 2017 |
| Population and density of the neighborhoods hosting the stations, outside Rome | https://www.tuttitalia.it/lazio/27-comuni/popolazione/ | 2019 |
| Pollution levels (as a rough indicator of traffic) | http://www.arpalazio.net/main/aria/sci/qa/misure/PM10.php | 2019 |

### Necessary imports and installations

In [ ]:
# installing geocoder, geopy and folium
!pip3 install geocoder --user
!pip3 install geopy --user
!pip3 install folium --user

# importing pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# importing requests for web page retrieval
import requests

# importing Beautiful soup for web page parsing
from bs4 import BeautifulSoup as BS

# importing matplotlib for map colors
import matplotlib.cm as cm
import matplotlib.colors as colors

# importing math for NaN processing
import math

# importing geocoder, Nominatim and folium for map generation
import geocoder 
from geopy.geocoders import Nominatim 
import folium

# importing KMeans for clustering
from sklearn.cluster import KMeans

In [ ]:
# installing xlwt for excel export
!pip3 install xlwt

### Loading Data

Loading the stations from the web page

In [ ]:
#loading stations from the web page

# source url
url = "http://www.rfi.it/rfi/LINEE-STAZIONI-TERRITORIO/Nelle-regioni/Lazio"

# performing the request
file = requests.get(url).text

Parsing the text with BeautifulSoup to retrieve the table

In [ ]:
# parsing data with Beautiful Soup
parsable_file = BS(file, 'lxml')

# retrieving the table
data_table_list = parsable_file.find_all('table')
data_table = data_table_list[1]

Converting the table into a list

In [ ]:
# converting the table into a list
list = pd.read_html(str(data_table), header=0)
list

Converting the list into a dataframe

In [ ]:
# converting the list into a dataframe
df_stations = list[0]
df_stations.head(200)

Dropping useless rows and columns and giving the dataframe its final shape

In [ ]:
# dropping the row with useless repetitions (STAZIONI(*))
df_stations = df_stations.drop([0])
df_stations = df_stations.drop([167])

# setting the proper row as column headers (Nome stazione/Fermata...)
df_stations.columns = df_stations.iloc[0]

# dropping the duplicate header
df_stations = df_stations.drop([1])

df_stations.head(200)

In [ ]:
# dropping the column reporting the company managing the station
df_stations = df_stations.drop('Network', axis=1)
df_stations.head()

In [ ]:
# saving stations to excel
df_stations.to_excel(r'stations.xls')

Loading Rome neighbourhoods from the webpage

In [ ]:
#loading neighbourhoods from the web page

# source url
url_neigh = "https://it.wikipedia.org/wiki/Municipi_di_Roma"

# performing the request
file_neigh = requests.get(url_neigh).text

Parsing the text with BeautifulSoup to retrieve the table

In [ ]:
# parsing data with Beautiful Soup
parsable_file = BS(file_neigh, 'lxml')

# retrieving the table
data_table_list = parsable_file.find_all('table')
data_table = data_table_list[0]

Converting the table into a list

In [ ]:
# converting the table into a list
list = pd.read_html(str(data_table), header=0)
list

Converting the list into a dataframe

In [ ]:
# converting the list into a dataframe
df_rome_neigh = list[0]
df_rome_neigh.head(200)

Deleting un-necessary columns

In [ ]:
df_rome_neigh = df_rome_neigh.drop(['Superficie(km²)', 'Presidente','Corrispondenza con Municipi 2001-2013e Circoscrizioni', 'Suddivisioni amministrative'], axis = 1)
df_rome_neigh.head()

In [ ]:
# saving Rome neighbours to excel
df_rome_neigh.to_excel(r'rome_neighbourhoods.xls')

Loading Lazio neighbourhoods from the webpage

In [ ]:
#loading Lazio neighbourhoods (municipi) from the web page

# source url
url_munic = "https://www.tuttitalia.it/lazio/27-comuni/popolazione/"

# performing the request
file_munic = requests.get(url_munic).text

Parsing the text with BeautifulSoup to retrieve the table

In [ ]:
# parsing data with Beautiful Soup
parsable_file = BS(file_munic, 'lxml')

# retrieving the table
data_table_list = parsable_file.find_all('table')
data_table = data_table_list[0]

Converting the table into a list

In [ ]:
# converting the table into a list
list = pd.read_html(str(data_table), header=0)
list

Converting the list into a dataframe

In [ ]:
# converting the list into a dataframe
df_rome_munic = list[0]
df_rome_munic.head(200)

Deleting un-necessary columns/rows and fixing province/city mixed up error

In [ ]:
# dropping un-necessary columns
df_rome_munic = df_rome_munic.drop(['Unnamed: 0', 'Superficiekm²','Altitudinem s.l.m.'], axis = 1)
df_rome_munic.head()

In [ ]:
# dropping un-necessary rows
df_rome_munic = df_rome_munic[:50]
df_rome_munic.head(60)

In [ ]:
def split_first_two(field):
    field = field[0:2]
    return field

def split_last_two(field):
    field = field[2:]
    return field


df_rome_munic['Provincia'] = df_rome_munic['Comune'].apply(split_first_two)
df_rome_munic['Comune'] = df_rome_munic['Comune'].apply(split_last_two)
df_rome_munic.head(50)

Saving Lazio neighbours to Excel

In [ ]:
# saving Lazio neighbours to excel
df_rome_munic.to_excel(r'lazio_neighbourhoods.xls')

Loading pollution from the excel

In [ ]:
#loading pollutions from the excel

# source filepath
filepath = "lazio_poll.xls"

# populating the dataframe
df_poll = pd.read_excel(filepath)

df_poll.head(50)

Dropping the un-necessary columns

In [ ]:
df_poll = df_poll.drop(['PM10 agosto'], axis = 1)
df_poll.head()

Saving pollutions to Excel

In [ ]:
# saving pollutions to excel
df_poll.to_excel(r'lazio_poll.xls')

## Methodology <a name="methodology"></a>

For each station, from the address will be derived the latitude and longitude, and those in turn will be used to retrieve the FourSquare data and venues. The match with population, density and pollution data will be done on the basis of the neighborhood hosting the station, using a closest distance function on longitude and latitude for joining tables.
The above data will be collected in a single dataframe, with a row for each station and columns reporting population, density, pollution and a set of FourSquare categories with the number of top venues occurring for each category.
The features will be standardized for better manipulation of the clustering algorithm, and then a clustering will be performed with 4 clusters, the same number of the classes used by RFI.
The original classes and the new clusters will then be compared, using both tables and maps, to check for correlations between the two.

Loading stations from the excel

In [ ]:
#loading stations from the excel

# source filepath
filepath = "stations.xls"

# populating the dataframe
df_stations = pd.read_excel(filepath)

df_stations.head(200)

Retrieving a list containing address (for control), Longitude and Latitude of each train station

In [ ]:
# retrieving longitude and latitude of each train station

# instantiating a geolocator
geolocator = Nominatim(user_agent="stat_explorer")
count = 0
# retrieving data for each station
columns_list = []

for address, city, name in zip(df_stations['Indirizzo'], df_stations['Comune/Località'], df_stations['Nome Stazione/fermata']):
    complete_address = address + ',' + city
    # passing the location to the geolocator
    location = geolocator.geocode(complete_address)
    if location is None:
        location = geolocator.geocode(str(name))  
        print('nome '+name)
    else: print('complete_address '+complete_address)
    # retrieving latitude and longitude from the geolocator
    
    
    print(location)
    print(count)
    count = count + 1
    latitude = location.latitude
    longitude = location.longitude
    display_name = location.address
    columns_list.append([latitude, longitude, display_name])

columns_list

Transforming the retrieved list in dataframe

In [ ]:
columns_df = pd.DataFrame(columns_list)
columns_df.columns=['latitude', 'longitude', 'address']
columns_df.head(200)

Adding coordinates to the original table

In [ ]:
df_stations_coord = pd.concat([df_stations, columns_df], axis=1, sort=False)
df_stations_coord.head(200)

Removing all un-necessary columns

In [ ]:
df_stations_coord = df_stations_coord.drop(['Unnamed: 0', 'Indirizzo', 'Comune/Località', 'Provincia', 'address'], axis = 1)
df_stations_coord.head()

Loading pollutions from excel

In [ ]:
#loading pollutions from the excel

# source filepath
filepath = "lazio_poll.xls"

# populating the dataframe
df_poll = pd.read_excel(filepath)

df_poll.head(200)

Retrieving a list containing address (for control), Longitude and Latitude of each pollution measuring station

In [ ]:
# retrieving longitude and latitude of each pollution station

# instantiating a geolocator
geolocator = Nominatim(user_agent="stat_explorer")
count = 0
# retrieving data for each station
columns_list = []

for address, city in zip(df_poll['Centralina'], df_poll['Provincia']):
    complete_address = address + ',' + city
    # passing the location to the geolocator
    location = geolocator.geocode(complete_address)
    if location is None:
        location = geolocator.geocode(str(address))  
        print('nome '+address)
    else: print('complete_address '+complete_address)
    # retrieving latitude and longitude from the geolocator
    
    
    print(location)
    print(count)
    count = count + 1
    latitude = location.latitude
    longitude = location.longitude
    display_name = location.address
    columns_list.append([latitude, longitude, display_name])

columns_list

Transforming the retrieved list in dataframe

In [ ]:
columns_df = pd.DataFrame(columns_list)
columns_df.columns=['latitude', 'longitude', 'address']
columns_df.head(200)

Adding coordinates to the original table

In [ ]:
df_poll_coord = pd.concat([df_poll, columns_df], axis=1, sort=False)
df_poll_coord.head(200)

Dropping un-necessary columns

In [ ]:
df_poll_coord = df_poll_coord.drop(['Provincia', 'address'], axis = 1)
df_poll_coord.head()

Loading neighbourhoods and populations from excel

In [ ]:
#loading neighbourhoods and populations from the excel

# source filepath
filepath = "locations_populations.xls"

# populating the dataframe
df_pop = pd.read_excel(filepath)

df_pop.head(200)

Retrieving a list containing address (for control), Longitude and Latitude of each neighbourhood

In [ ]:
# retrieving longitude and latitude of each neighbourhood

# instantiating a geolocator
geolocator = Nominatim(user_agent="stat_explorer")
count = 0
# retrieving data for each neighbourhood
columns_list = []

for address, provincia in zip(df_pop['address'], df_pop['Provincia']):
    complete_address = address + ',' + city
    # passing the location to the geolocator
    location = geolocator.geocode(complete_address)
    if location is None:
        location = geolocator.geocode(str(address))  
        print('nome '+address)
    else: print('complete_address '+complete_address)
    # retrieving latitude and longitude from the geolocator
    
    
    print(location)
    print(count)
    count = count + 1
    latitude = location.latitude
    longitude = location.longitude
    display_name = location.address
    columns_list.append([latitude, longitude, display_name])

columns_list

Transforming the retrieved list in dataframe

In [ ]:
columns_df = pd.DataFrame(columns_list)
columns_df.columns=['latitude', 'longitude', 'address']
columns_df.head(200)

Adding coordinates to the original table

In [ ]:
df_pop_coord = pd.concat([df_pop, columns_df], axis=1, sort=False)
df_pop_coord.head(200)

Dropping un-necessary columns

In [ ]:
df_pop_coord = df_pop_coord.drop(['Unnamed: 0','Provincia', 'address'], axis = 1)
df_pop_coord.head()

Merging together: stations, pollution and population data

In [ ]:
# assigning at each station the correct pollution and population values using latitude and longitude

# importing the geopy function calculating distance of two points given the coordinates
import geopy.distance

# maximum distance between points on Earth
max_distance = 20000

# this function returns PM10 average and number of days above max level for given coordinates
def find_pollution(latitude, longitude):

    # station coordinates
    stat_coord = (latitude, longitude)
    
    temp_distance = max_distance
    
    pollution = []
    
    for lat, long, pm10, bre in zip(df_poll_coord['latitude'], df_poll_coord['longitude'], df_poll_coord['PM10 media'], df_poll_coord['Sforamenti']):
        # pollution station coordinates
        poll_coord = (lat, long)
        # distance between points according to the Vincenty's formula
        distance = geopy.distance.vincenty(stat_coord, poll_coord).km
        if distance < temp_distance: 
            temp_distance = distance
            temp_PM10 = pm10
            temp_sforamenti = bre
    
    temp_PM10  
    temp_sforamenti
    
    return temp_PM10, temp_sforamenti

# this function returns population total and density for given coordinates
def find_population(latitude, longitude):

    # station coordinates
    stat_coord = (latitude, longitude)
    
    temp_distance = max_distance
    
    population = []
    for lat, long, pop, dens in zip(df_pop_coord['latitude'], df_pop_coord['longitude'], df_pop_coord['Popolazione(ab)'], df_pop_coord['Densità(ab/km²)']):
        # neighbourhood coordinates
        pop_coord = (lat, long)
        # distance between points according to the Vincenty's formula
        distance = geopy.distance.vincenty(stat_coord, pop_coord).km
        if distance < temp_distance: 
            temp_distance = distance
            temp_population = pop
            temp_density = dens
    
    temp_population  
    temp_density
    
    return temp_population, temp_density

columns_list = []
count = 0

# finding the pollution and population values for each station
for latitude, longitude in zip(df_stations_coord['latitude'], df_stations_coord['longitude']):
    
    # finding the pollution values corresponding to the station
    pm10, overcoming = find_pollution(latitude, longitude)
    
    # finding the population values corresponding to the station
    population, density = find_population(latitude, longitude)
    
    count = count + 1
    print(pm10)
    print(overcoming)
    print(population)
    print(density)
    print(latitude) 
    print(longitude) 
    columns_list.append([pm10, overcoming, population, density])

columns_list

Creating a dataframe from the list

In [ ]:
columns_df = pd.DataFrame(columns_list)
columns_df.columns=['PM10', 'overcomings', 'population', 'density']
columns_df.head(200)

Adding data to the original table

In [ ]:
df_stations_coord_pop_poll = pd.concat([df_stations_coord, columns_df], axis=1, sort=False)
df_stations_coord_pop_poll.head(200)

Extracting the venues for each station, by category

Storing my FourSquare credentials in variables

In [ ]:
# my foursquare credentials
CLIENT_ID = 'GJAYQ42Z4F1LCF2G2QRREQUEF5B0YD0HC1CIGM3ZXEZWYVWD' # my Foursquare ID
CLIENT_SECRET = '3NG0CMKA3LYWVW3ZUXXLDMF33DNJUKHNUJPMVGPMKTTFSIM3' # my Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Retrieving the 10 high-level categories of venues

In [ ]:
# building the url requesting all the categories
categories_url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)

# executing the request
results = requests.get(categories_url).json()

number_high_categories = len(results['response']['categories'])
print('number of high-level categories: '+ str(number_high_categories))

In [ ]:
#inizializing the list
categories_list = []  

# storing the high-level categories into a list
for i in range(0,number_high_categories):
    id = results['response']['categories'][i]['id']
    name = results['response']['categories'][i]['name']
    print(id)
    print(name)
    categories_list.append([name, id])

#showing the list content
print(categories_list)

Adding columns to the stations dataframe to store the venues data

In [ ]:
df_stations_venues = df_stations_coord_pop_poll.copy()

# for each categoy, a new empty column is created
for categories in categories_list:
    # the empty column creation
    df_stations_venues[categories[0]] = 0
df_stations_venues.head(200)

Retrieving the venues count for each station

In [ ]:
# limiting the calls to stay in the free account

# a function returning the number of venues belonging to a category and close to the given lat and long
def get_total_venues_count(latitude, longitude, categoryId, radius):
    # latitude and longitude are joined in a single coordinates field
    latlong = str(latitude) + ',' + str(longitude)
    # the url is built
    explore_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={}&radius={}&categoryId={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION,
                latlong,
                radius,
                categoryId)
        
    # the request is executed, asking for the totalResults, that is the needed venues count
    return requests.get(explore_url).json()['response']['totalResults']

In [ ]:
# retrieving the venues count
count = 1
for i, row in df_stations_venues.iterrows():
    for category in categories_list:
        df_stations_venues.loc[i, category[0]] = get_total_venues_count(df_stations_venues.latitude.iloc[i],df_stations_venues.longitude.iloc[i], categoryId = category[1], radius = 1500)

In [ ]:
# saving the venues to excel
df_stations_venues.to_excel(r'stations_venues.xls')

## Analysis <a name="analysis"></a>

Stations with full data are loaded and displayed

In [ ]:
#loading stations from the excel

# source filepath
filepath = "stations_venues.xls"

# populating the dataframe
df_stations = pd.read_excel(filepath)
df_stations = df_stations.drop('Unnamed: 0', axis=1)
df_stations.head(200)

Data structure is checked

In [ ]:
df_stations.info()

Data statistic is explored

In [ ]:
df_stations.describe()

Since Events are non significant, the columns is dropped.

In [ ]:
df_stations = df_stations.drop(['Event'], axis=1)

In [ ]:
df_stations.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 10))
plt.xticks(rotation='vertical')
sns.boxplot

df_stations_copy = df_stations.drop(['population', 'density','latitude','longitude'], axis=1)
ax = sns.boxplot(data = df_stations_copy)
ax.set_ylabel('Count of occurrences', fontsize=25)
ax.set_xlabel('Attributes', fontsize=25)
ax.tick_params(labelsize=20)
plt.xticks(rotation=45, ha='right')

plt.show()

The most represented venues are Food, Professional and Shops, and all venues present significant upper outliers. PM10 and overcomings are more evenly distributed.

The data is now standardized as a preparation for clustering.

In [ ]:
# importing MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

#getting the values to be standardized (excluding name and category)
X = df_stations.values[:,4:]
# applying scaler
scaled_dataset = MinMaxScaler().fit_transform(X)

In [ ]:
# removing the no-more used Event element from the categories list
categories_list.remove(['Event', '4d4b7105d754a06373d81259'])
categories_list

In [ ]:
# transforming the scaled dataset in dataframe
scaled_dataframe = pd.DataFrame(scaled_dataset)
# setting the column names again
scaled_dataframe.columns = ['PM10','overcomings', 'population', 'density']+[category[0] for category in categories_list]
scaled_dataframe.head()

In [ ]:
# plotting as before, but with scaled data using pop and density also
plt.figure(figsize=(20, 10))
plt.xticks(rotation='vertical')
sns.boxplot

ax = sns.boxplot(data = scaled_dataframe)
ax.set_ylabel('Count of occurrences', fontsize=25)
ax.set_xlabel('Attributes', fontsize=25)
ax.tick_params(labelsize=20)
plt.xticks(rotation=45, ha='right')

plt.show()

Clustering on 4 clusters, as many as the original categories

In [ ]:
# number of clusters
clusters = 4

# performing the k-means clustering
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(scaled_dataframe)

# retrieving the labels
kmeans_labels = kmeans.labels_

# Change label numbers so they go from highest scores to lowest
replace_labels = {0:2, 1:0, 2:3, 3:1}
for i in range(len(kmeans_labels)):
    kmeans_labels[i] = replace_labels[kmeans_labels[i]]

df_stations_clusters_labels = df_stations.copy()
df_stations_clusters_labels['Cluster'] = kmeans_labels
stations_clusters_minmax_df = scaled_dataframe.copy()
stations_clusters_minmax_df['Cluster'] = kmeans_labels
stations_clusters_minmax_df['Nome Stazione/fermata'] = df_stations['Nome Stazione/fermata']
stations_clusters_minmax_df['latitude'] = df_stations['latitude']
stations_clusters_minmax_df['longitude'] = df_stations['longitude']

Showing the characteristics of each cluster

In [ ]:
# importing ticker
import matplotlib.ticker as ticker

# creating a figure
fig, axes = plt.subplots(1,clusters, figsize=(20, 10), sharey=True)

# setting y label
axes[0].set_ylabel('Count of attributes (relative)', fontsize=25)

# building the boxes for each cluster
for k in range(clusters):
    #Set same y axis limits
    axes[k].set_ylim(0,1.1)
    axes[k].xaxis.set_label_position('top')
    axes[k].set_xlabel('Cluster ' + str(k), fontsize=25)
    axes[k].tick_params(labelsize=20)
    plt.sca(axes[k])
    plt.xticks(rotation='vertical')
    # filling the box with boxplots
    sns.boxplot(data = stations_clusters_minmax_df[stations_clusters_minmax_df['Cluster'] == k].drop('Cluster',1), ax=axes[k])

plt.show()

The first cluster shows an excellent presence in all categories, while the second is significantly below. 
The third and fourth ones have smaller indicators and are differentiated mostly by distribution of venues
and population/pollution differences.

In [ ]:
# saving results to excel
df_stations_clusters_labels.to_excel(r'stations_labeled.xls')

In [ ]:
#loading labeled stations from the excel

# source filepath
filepath = "stations_labeled.xls"

# populating the dataframe
df_stations = pd.read_excel(filepath)
df_stations = df_stations.drop('Unnamed: 0', axis=1)
df_stations.head(200)

In [ ]:
# mapping categories to numbers for easier confrontation
mapping = {'PLATINUM': 0, 'GOLD': 1, 'SILVER': 2, 'BRONZE': 3}
df_stations = df_stations.replace({'Categoria': mapping})
df_stations.to_excel(r'stations_labeled.xls')
df_stations.head(200)

In [ ]:
# checking for the correctly labeled stations
df_correctly_labeled = df_stations[df_stations['Categoria'] == df_stations['Cluster']]
print(str(len(df_correctly_labeled)) + ' stations out of 165 were correctly labeled')

In [ ]:
# exploring the uncorrectly labeled stations
df_uncorrectly_labeled = df_stations[df_stations['Categoria'] != df_stations['Cluster']]
df_uncorrectly_labeled = df_uncorrectly_labeled[['Nome Stazione/fermata', 'Categoria', 'Cluster']]
print(str(len(df_uncorrectly_labeled)) + ' stations out of 165 were uncorrectly labeled')
df_uncorrectly_labeled.head(100)

84% of stations matched the original RFI classification.
16% of stations fell into the category immediately below or above.

Showing the map with original RFI classification

In [ ]:
# Rome coordinates

latitude = 41.9028
longitude = 12.4964

# creating map
map_rfi = folium.Map(location=[latitude, longitude], zoom_start=9)

# setting color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)] 
colors_array = cm.rainbow(np.linspace(0, 1, len(ys))) 
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, categoria in zip(df_stations['latitude' ], df_stations['longitude'], df_stations['Nome Stazione/fermata'], df_stations['Categoria']):
    label = folium.Popup(str(poi) + ' Categoria ' + str(categoria), parse_html=True)
    if math.isnan(categoria): categoria=5
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label, 
        color=rainbow[int(categoria-1)],
        fill=True,
        fill_color=rainbow[int(categoria-1)],
        fill_opacity=0.7).add_to(map_rfi)
map_rfi
 

Showing the map with original cluster classification

In [ ]:
# Rome coordinates

latitude = 41.9028
longitude = 12.4964

# creating map
map_rfi = folium.Map(location=[latitude, longitude], zoom_start=9)

# setting color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)] 
colors_array = cm.rainbow(np.linspace(0, 1, len(ys))) 
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, categoria in zip(df_stations['latitude' ], df_stations['longitude'], df_stations['Nome Stazione/fermata'], df_stations['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(categoria), parse_html=True)
    if math.isnan(categoria): categoria=5
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label, 
        color=rainbow[int(categoria-1)],
        fill=True,
        fill_color=rainbow[int(categoria-1)],
        fill_opacity=0.7).add_to(map_rfi)
map_rfi

## Results and Discussion <a name="results"></a>

Goal of the project was to use data gathered from Foursquare and other sources to classify the stations independently and understand if some stations should deserve a different status from the one given by the network manager (RFI = Rete Ferroviaria Italiana), according to the characteristics of the nearby area.

RFI, in fact, arranged the stations into four categories: Platinum, Gold, Silver and Bronze, depending on the level of services offered in each stations.

The list of the 165 train stations of the Italian region named Lazio was successfully websourced, as well as their addresses. 

Two more datasets were websourced:

1. a list of pollution measuring stations, with average yearly emissions (PM10) and number of days when the maximum threshold has been trespassed (overcomings); these data were used mainly as indicators of traffic;
2. a list of all main neighbourhoods in Lazio, with the overall population and density.

Using Nominatim, the address of the three POIs above were converted into geographical coordinates (longitude and latitude).

After that, a function based on the Vincenty geodesic distance was employed, to associate the closest pollution station and the closest neighbourhood, as well as their data, to each train station.

The final step of the data collection used FourSquare to gather, for each station, the number of venues in a range of 1500 meters, arranged in the FourSquare top 10 categories, that are:

- Arts & Entertainment
- College & University
- Event
- Food
- Nightlife Spot
- Outdoors & Recreation
- Professional & Other Places
- Residence
- Shop & Service
- Travel & Transport

After scaling the data, the K-Means clustering was applied, using a 4 clusters, the same number of the RFI categories.

The visual analysis of the clusters showed a first cluster with an excellent presence in all categories, with the second significantly below. The third and fourth ones had smaller quantities in all venue indicators and were differentiated mostly by distribution of venues and population/pollution differences.

84% of stations matched the original RFI classification. 16% of stations fell into the category immediately below or above. This overall agreement between original categories and computed clusters was shown by visual maps as well.

The stations that were differently classified fell mostly into 2 categories:

1. stations in the centre of Rome, with a venue rich neighbourhood, promoted to the above category (such as Trastevere, S. Pietro, Aurelia, Quattro Venti, Balduina, Monte Mario)
2. stations playing a crucial role as a node in the transportation network, but with poorer neighbourhood, that were demoted to the category immediately below (such as Tiburtina, Fiumicino, Ciampino, Civitavecchia)

Therefore, nothwithstanding the fact that FourSquare Data is un-balanced, with some categories, such as food, over-represented, the clustering made quite sense, with the stations distribution matching broadly the investments made by the network manager on each station, as reflected by the category level.

The non matching labels made sense as well, showing that some stations deserved a higher level than the one suggested by the nearby area, given the strategic position (a close-by airport or harbour), while others should deserve a promotion considering the amount of venues and activities in the nearby area.

## Conclusion <a name="conclusion"></a>

The project made use of websourced data and a non-commercial FourSquare account.

Notwithstanding these limited resources, it proved strong insights into the areas surrounding each station, supporting the choices made by the rail network manager and giving some valuable suggestion.

Further applications could be:

1. exploring the predicted impact of a station promotion/demotion on its reviews;
2. exploring the correlation of check-ins and reviews to other parameters, like availability of certain venues, such as parking or bus stops or restaurants, or the lack of others, such as coworking places or professional structures;
3. the creation of a recommendation system able to advice citizens and tourists about the nearby venues of each station, depending on their needs and past choices of similar users.

The project could be extended to the whole Italian network as well.